# Initial EDA - Cryptocurrency Market Surveillance

**Tujuan:** Eksplorasi awal struktur data dari Binance API

**Data Source:** Binance REST API (`api4.binance.com`)

**Trading Pairs:** BTCUSDT, ETHUSDT, BNBUSDT

## 1. Setup & Imports

In [ ]:
import requests
import pandas as pd
import numpy as np
import json
from datetime import datetime

BASE_URL = 'https://api4.binance.com'
TRADING_PAIRS = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']

print('Setup complete!')
print(f'Pandas version: {pd.__version__}')
print(f'NumPy version: {np.__version__}')

## 2. Fetch Sample Trade Data

In [ ]:
def fetch_recent_trades(symbol: str, limit: int = 100) -> pd.DataFrame:
    """Fetch recent trades dari Binance REST API."""
    endpoint = f'{BASE_URL}/api/v3/trades'
    params = {'symbol': symbol, 'limit': limit}
    
    response = requests.get(endpoint, params=params)
    response.raise_for_status()
    
    df = pd.DataFrame(response.json())
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    df['price'] = df['price'].astype(float)
    df['qty'] = df['qty'].astype(float)
    df['quoteQty'] = df['quoteQty'].astype(float)
    
    return df

# Test fetch BTCUSDT
btc_trades = fetch_recent_trades('BTCUSDT', limit=100)
print(f'Shape: {btc_trades.shape}')
print(f'Columns: {btc_trades.columns.tolist()}')
btc_trades.head()

## 3. Fetch OHLCV Klines Data

In [ ]:
def fetch_klines(symbol: str, interval: str = '1m', limit: int = 100) -> pd.DataFrame:
    """Fetch OHLCV klines dari Binance REST API."""
    endpoint = f'{BASE_URL}/api/v3/klines'
    params = {'symbol': symbol, 'interval': interval, 'limit': limit}
    
    response = requests.get(endpoint, params=params)
    response.raise_for_status()
    
    columns = [
        'open_time', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_volume', 'trades_count',
        'taker_buy_base', 'taker_buy_quote', 'ignore'
    ]
    
    df = pd.DataFrame(response.json(), columns=columns)
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = df[col].astype(float)
    
    return df

btc_klines = fetch_klines('BTCUSDT', interval='1m', limit=100)
print(f'Shape: {btc_klines.shape}')
btc_klines.head()

## 4. Basic Statistics & Data Quality Check

In [ ]:
print('=== BTCUSDT Klines Basic Statistics ===')
print(btc_klines[['open', 'high', 'low', 'close', 'volume']].describe())

print('\n=== Missing Values ===')
print(btc_klines.isnull().sum())

print('\n=== Data Types ===')
print(btc_klines.dtypes)

## 5. Volatility Calculation (Rolling Std)

In [ ]:
btc_klines['returns'] = btc_klines['close'].pct_change()
btc_klines['volatility_5m'] = btc_klines['returns'].rolling(5).std()
btc_klines['volatility_15m'] = btc_klines['returns'].rolling(15).std()

print('=== Volatility Statistics ===')
print(btc_klines[['volatility_5m', 'volatility_15m']].describe())